# Beach Crowd Prediction — LSTM, TFT, TiDE + Optuna Optimization

This notebook compares:
1. **Sklearn baseline models** (quick comparison)
2. **NeuralForecast models**: LSTM, TFT, TiDE
3. **Optuna-optimized models**: LSTM, XGBoost, CatBoost, LightGBM

**Model Selection Rationale:**
- **TFT**: Best for new beach prediction via static features (lat/lon/capacity)
- **TiDE**: Fast, lightweight encoder-decoder with exogenous support
- **LSTM**: Classic baseline for sequential data

In [ ]:
# === PATHS ===
CACHE_DIR = "cache/predictions"
COUNTING_MODEL = "bayesian_vgg19"
SAVE_DIR = "models/optuna_comparison"

# === SAMPLING ===
SAMPLE_FRAC = 1.0
MAX_BEACHES = None

# === MODEL PARAMETERS ===
MAX_STEPS = 500
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
INPUT_SIZE = 24

# === TIME ===
NIGHT_START = 20
NIGHT_END = 6

# === OPTUNA ===
OPTUNA_TRIALS = 30
OPTUNA_TIMEOUT = 300

# === FLAGS ===
RUN_SKLEARN = True
RUN_NEURALFORECAST = True
RUN_OPTUNA = True

In [ ]:
import subprocess, sys
pkgs = ["neuralforecast", "xgboost", "lightgbm", "catboost", "utilsforecast", "optuna"]
for pkg in pkgs:
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "-q"])
print("Packages installed")

In [ ]:
import json, time, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Lasso
import torch

warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except: HAS_XGB = False

try:
    from lightgbm import LGBMRegressor
    HAS_LGBM = True
except: HAS_LGBM = False

try:
    from catboost import CatBoostRegressor
    HAS_CATBOOST = True
except: HAS_CATBOOST = False

try:
    from neuralforecast import NeuralForecast
    from neuralforecast.models import LSTM, TFT, TiDE
    from neuralforecast.losses.pytorch import MAE
    HAS_NF = True
except Exception as e:
    print(f"NeuralForecast error: {e}")
    HAS_NF = False

if torch.cuda.is_available():
    ACCELERATOR = 'gpu'
    DEVICES = 1
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    ACCELERATOR = 'mps'
    DEVICES = 1
else:
    ACCELERATOR = 'cpu'
    DEVICES = 1

print(f"Accelerator: {ACCELERATOR}")
print(f"XGB: {HAS_XGB}, LGBM: {HAS_LGBM}, CatBoost: {HAS_CATBOOST}, NF: {HAS_NF}")

In [ ]:
def calc_metrics(y_true, y_pred, max_count):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    rel_mae = (mae / max_count) * 100 if max_count > 0 else 0
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'RelMAE': rel_mae}

def eval_per_beach(df, y_pred, beach_col='unique_id'):
    results = []
    for b in df[beach_col].unique():
        mask = df[beach_col] == b
        if mask.sum() < 3:
            continue
        y_true = df.loc[mask, 'y'].values if 'y' in df.columns else df.loc[mask, 'count'].values
        y_p = y_pred[mask.values] if hasattr(mask, 'values') else y_pred[mask]
        max_count = y_true.max()
        m = calc_metrics(y_true, y_p, max_count)
        m['camera'] = b
        m['max_count'] = max_count
        m['n'] = mask.sum()
        results.append(m)
    return pd.DataFrame(results)

## Load and Prepare Data

In [ ]:
def load_cache(cache_dir, model):
    cache_path = Path(cache_dir) / model
    records = []
    for jf in cache_path.rglob("*.json"):
        try:
            with open(jf) as f:
                r = json.load(f)
            if 'error' not in r:
                records.append(r)
        except: pass
    
    rows = []
    for r in records:
        row = {
            'beach': r.get('beach') or r.get('beach_folder'),
            'beach_folder': r.get('beach_folder'),
            'datetime': r.get('datetime'),
            'count': r.get('count')
        }
        for k, v in r.get('weather', {}).items():
            row[k] = v
        rows.append(row)
    
    df = pd.DataFrame(rows)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values('datetime').reset_index(drop=True)
    return df

df_raw = load_cache(CACHE_DIR, COUNTING_MODEL)
print(f"Loaded: {len(df_raw)} rows, {df_raw['beach'].nunique()} beaches")

In [ ]:
EXCLUDE = ['livecampro/001', 'livecampro/011', 'livecampro/018', 'livecampro/021',
    'livecampro/030', 'livecampro/039', 'livecampro/070', 'MultimediaTres/PortAndratx',
    'SeeTheWorld/mallorca_pancam', 'skyline/es-pujols']
EXCLUDE_PREFIX = ['ibred', 'ClubNauticSoller', 'Guenthoer', 'youtube']

before = len(df_raw)
df_raw = df_raw[~df_raw['beach_folder'].isin(EXCLUDE)]
for p in EXCLUDE_PREFIX:
    df_raw = df_raw[~df_raw['beach_folder'].str.startswith(p, na=False)]
print(f"Filtered: {before} -> {len(df_raw)}")

In [ ]:
if SAMPLE_FRAC < 1.0:
    df_raw = df_raw.sample(frac=SAMPLE_FRAC, random_state=42).sort_values('datetime').reset_index(drop=True)

if MAX_BEACHES:
    top = df_raw['beach'].value_counts().head(MAX_BEACHES).index.tolist()
    df_raw = df_raw[df_raw['beach'].isin(top)].reset_index(drop=True)

print(f"Final: {len(df_raw)} rows, {df_raw['beach'].nunique()} beaches")

In [ ]:
df = df_raw.copy()
df['hour'] = df['datetime'].dt.hour
df['day_of_week'] = df['datetime'].dt.dayofweek
df['month'] = df['datetime'].dt.month
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
df['is_summer'] = df['month'].isin([6, 7, 8]).astype(int)
df['is_night'] = ((df['hour'] >= NIGHT_START) | (df['hour'] <= NIGHT_END)).astype(int)

WEATHER_COLS = [c for c in df.columns if c.startswith('ae_') or c.startswith('om_')]
TEMPORAL_COLS = ['hour', 'day_of_week', 'month', 'is_weekend', 'is_summer', 'is_night']
ALL_FEATURES = WEATHER_COLS + TEMPORAL_COLS

df = df.dropna(subset=ALL_FEATURES + ['count']).reset_index(drop=True)
good = df.groupby('beach')['count'].max()
good = good[good > 20].index.tolist()
df = df[df['beach'].isin(good)].reset_index(drop=True)

print(f"After cleaning: {len(df)} rows, {len(good)} beaches")
print(f"Features: {len(ALL_FEATURES)}")

In [ ]:
# Create Night=0 dataset (best performer in previous experiments)
ds_night0 = df.copy()
ds_night0.loc[ds_night0['is_night'] == 1, 'count'] = 0.0

print(f"Night0 dataset: {len(ds_night0)} rows")
print(f"  Night rows (set to 0): {(ds_night0['is_night'] == 1).sum()}")
print(f"  Day rows: {(ds_night0['is_night'] == 0).sum()}")

In [ ]:
def split_data(df, train_frac=0.7, val_frac=0.15):
    n = len(df)
    t1 = int(n * train_frac)
    t2 = int(n * (train_frac + val_frac))
    return df.iloc[:t1], df.iloc[t1:t2], df.iloc[t2:]

train, val, test = split_data(ds_night0)
print(f"Train: {len(train)}, Val: {len(val)}, Test: {len(test)}")

## Sklearn Baseline Models

In [ ]:
all_results = []
all_beach_results = []

X_train = pd.concat([train, val])[ALL_FEATURES]
y_train = pd.concat([train, val])['count']
X_test = test[ALL_FEATURES]
y_test = test['count']

sklearn_models = {
    'Lasso': Lasso(alpha=0.1),
    'RandomForest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1),
}
if HAS_XGB:
    sklearn_models['XGBoost'] = XGBRegressor(n_estimators=200, max_depth=6, random_state=42, n_jobs=-1, verbosity=0)
if HAS_LGBM:
    sklearn_models['LightGBM'] = LGBMRegressor(n_estimators=200, max_depth=6, random_state=42, n_jobs=-1, verbose=-1)
if HAS_CATBOOST:
    sklearn_models['CatBoost'] = CatBoostRegressor(n_estimators=200, max_depth=6, random_state=42, verbose=0)

if RUN_SKLEARN:
    print("=" * 60)
    print("SKLEARN BASELINE")
    print("=" * 60)
    
    for name, model in sklearn_models.items():
        t0 = time.time()
        model.fit(X_train, y_train)
        y_pred = np.clip(model.predict(X_test), 0, None)
        elapsed = time.time() - t0
        
        m = calc_metrics(y_test.values, y_pred, y_test.max())
        beach_df = eval_per_beach(test, y_pred, 'beach')
        avg_rel = beach_df['RelMAE'].mean()
        
        all_results.append({
            'Model': name, 'Type': 'Sklearn',
            'MAE': m['MAE'], 'RMSE': m['RMSE'], 'R2': m['R2'],
            'AvgRelMAE': avg_rel, 'Time': elapsed
        })
        print(f"  {name:15s} | {elapsed:5.1f}s | MAE={m['MAE']:.1f} | RelMAE={avg_rel:.1f}% | R2={m['R2']:.3f}")

## NeuralForecast Models: LSTM, TFT, TiDE

**Model Comparison:**

| Model | Strengths | Best For |
|-------|-----------|----------|
| **LSTM** | Classic baseline, proven | Sequential patterns |
| **TFT** | Static features, interpretable attention | New beach prediction |
| **TiDE** | Fast, lightweight, exogenous support | Production speed |

In [ ]:
from utilsforecast.preprocessing import fill_gaps

def to_nf_format(df, id_col='beach_folder'):
    cols = ['datetime', id_col, 'count'] + ALL_FEATURES
    cols = [c for c in cols if c in df.columns]
    nf_df = df[cols].copy()
    nf_df = nf_df.rename(columns={'datetime': 'ds', id_col: 'unique_id', 'count': 'y'})
    return nf_df

def prepare_nf_data(train_df, test_df, freq='h'):
    nf_train = to_nf_format(train_df)
    nf_test = to_nf_format(test_df)
    
    # Deduplicate
    nf_train = nf_train.groupby(['unique_id', 'ds']).mean(numeric_only=True).reset_index()
    nf_test = nf_test.groupby(['unique_id', 'ds']).mean(numeric_only=True).reset_index()
    
    # Fill gaps
    nf_train = fill_gaps(nf_train, freq=freq)
    nf_test = fill_gaps(nf_test, freq=freq)
    
    # Interpolate
    for col in nf_train.select_dtypes(include=[np.number]).columns:
        nf_train[col] = nf_train.groupby('unique_id')[col].transform(
            lambda x: x.interpolate(method='linear').ffill().bfill()
        )
        nf_test[col] = nf_test.groupby('unique_id')[col].transform(
            lambda x: x.interpolate(method='linear').ffill().bfill()
        )
    
    # Keep common series
    common = set(nf_train['unique_id'].unique()) & set(nf_test['unique_id'].unique())
    nf_train = nf_train[nf_train['unique_id'].isin(common)].reset_index(drop=True)
    nf_test = nf_test[nf_test['unique_id'].isin(common)].reset_index(drop=True)
    
    return nf_train, nf_test, list(common)

train_val = pd.concat([train, val])
nf_train, nf_test, series_ids = prepare_nf_data(train_val, test)
print(f"NF Data: train={len(nf_train)}, test={len(nf_test)}, series={len(series_ids)}")

In [ ]:
def get_nf_models(hist_exog, horizon):
    common = dict(
        h=horizon,
        input_size=INPUT_SIZE,
        max_steps=MAX_STEPS,
        early_stop_patience_steps=-1,
        learning_rate=LEARNING_RATE,
        batch_size=BATCH_SIZE,
        scaler_type='robust',
        random_seed=42,
        accelerator=ACCELERATOR,
        devices=DEVICES,
        loss=MAE(),
    )
    
    return [
        ('LSTM', LSTM(
            hist_exog_list=hist_exog,
            encoder_hidden_size=64,
            encoder_n_layers=2,
            **common
        )),
        ('TFT', TFT(
            hist_exog_list=hist_exog,
            hidden_size=64,
            n_head=4,
            **common
        )),
        ('TiDE', TiDE(
            hist_exog_list=hist_exog,
            hidden_size=128,
            decoder_output_dim=16,
            **common
        )),
    ]

print("NF Models: LSTM, TFT, TiDE")

In [ ]:
if RUN_NEURALFORECAST and HAS_NF:
    nf_all = pd.concat([nf_train, nf_test]).sort_values(['unique_id', 'ds']).reset_index(drop=True)
    test_horizon = nf_test.groupby('unique_id').size().min()
    
    # Use smaller horizon for faster training
    horizon = min(72, test_horizon)
    
    print("=" * 60)
    print(f"NEURALFORECAST (horizon={horizon})")
    print("=" * 60)
    
    for model_name, model in get_nf_models(ALL_FEATURES, horizon=horizon):
        print(f"\n  {model_name}...")
        try:
            t0 = time.time()
            nf = NeuralForecast(models=[model], freq='h')
            
            # Use cross_validation for proper evaluation
            cv_results = nf.cross_validation(
                df=nf_all,
                n_windows=1,
                step_size=horizon,
            )
            elapsed = time.time() - t0
            
            pred_col = [c for c in cv_results.columns if c not in ['unique_id', 'ds', 'cutoff', 'y']][0]
            
            y_true = cv_results['y'].values
            y_pred = np.clip(cv_results[pred_col].values, 0, None)
            
            m = calc_metrics(y_true, y_pred, y_true.max())
            
            eval_df = cv_results.copy()
            eval_df['beach'] = eval_df['unique_id']
            beach_df = eval_per_beach(eval_df, y_pred, 'beach')
            avg_rel = beach_df['RelMAE'].mean() if len(beach_df) > 0 else np.nan
            
            all_results.append({
                'Model': model_name, 'Type': 'NeuralForecast',
                'MAE': m['MAE'], 'RMSE': m['RMSE'], 'R2': m['R2'],
                'AvgRelMAE': avg_rel, 'Time': elapsed
            })
            print(f"    {elapsed:.1f}s | MAE={m['MAE']:.1f} | RelMAE={avg_rel:.1f}% | R2={m['R2']:.3f}")
            
        except Exception as e:
            print(f"    ERROR: {e}")
            import traceback
            traceback.print_exc()

## Optuna Hyperparameter Optimization

Optimize:
- **XGBoost**: n_estimators, max_depth, learning_rate, subsample, colsample_bytree
- **LightGBM**: n_estimators, num_leaves, learning_rate, feature_fraction, bagging_fraction
- **CatBoost**: iterations, depth, learning_rate, l2_leaf_reg
- **LSTM** (via NeuralForecast): hidden_size, n_layers, learning_rate, dropout

In [ ]:
def create_xgb_objective(X_train, y_train, X_val, y_val):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
            'random_state': 42,
            'n_jobs': -1,
            'verbosity': 0,
        }
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        return mean_absolute_error(y_val, y_pred)
    return objective

def create_lgbm_objective(X_train, y_train, X_val, y_val):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'num_leaves': trial.suggest_int('num_leaves', 20, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
            'random_state': 42,
            'n_jobs': -1,
            'verbose': -1,
        }
        model = LGBMRegressor(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        return mean_absolute_error(y_val, y_pred)
    return objective

def create_catboost_objective(X_train, y_train, X_val, y_val):
    def objective(trial):
        params = {
            'iterations': trial.suggest_int('iterations', 100, 1000),
            'depth': trial.suggest_int('depth', 4, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
            'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
            'random_strength': trial.suggest_float('random_strength', 1e-8, 10.0, log=True),
            'border_count': trial.suggest_int('border_count', 32, 255),
            'random_state': 42,
            'verbose': 0,
        }
        model = CatBoostRegressor(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        return mean_absolute_error(y_val, y_pred)
    return objective

In [ ]:
if RUN_OPTUNA:
    X_tr = train[ALL_FEATURES]
    y_tr = train['count']
    X_va = val[ALL_FEATURES]
    y_va = val['count']
    
    best_models = {}
    
    print("=" * 60)
    print(f"OPTUNA OPTIMIZATION (trials={OPTUNA_TRIALS}, timeout={OPTUNA_TIMEOUT}s)")
    print("=" * 60)
    
    # XGBoost
    if HAS_XGB:
        print("\n  XGBoost...")
        study = optuna.create_study(direction='minimize')
        study.optimize(
            create_xgb_objective(X_tr, y_tr, X_va, y_va),
            n_trials=OPTUNA_TRIALS,
            timeout=OPTUNA_TIMEOUT,
            show_progress_bar=True
        )
        best_models['XGBoost_Optuna'] = study.best_params
        print(f"    Best MAE: {study.best_value:.2f}")
        print(f"    Best params: {study.best_params}")
    
    # LightGBM
    if HAS_LGBM:
        print("\n  LightGBM...")
        study = optuna.create_study(direction='minimize')
        study.optimize(
            create_lgbm_objective(X_tr, y_tr, X_va, y_va),
            n_trials=OPTUNA_TRIALS,
            timeout=OPTUNA_TIMEOUT,
            show_progress_bar=True
        )
        best_models['LightGBM_Optuna'] = study.best_params
        print(f"    Best MAE: {study.best_value:.2f}")
        print(f"    Best params: {study.best_params}")
    
    # CatBoost
    if HAS_CATBOOST:
        print("\n  CatBoost...")
        study = optuna.create_study(direction='minimize')
        study.optimize(
            create_catboost_objective(X_tr, y_tr, X_va, y_va),
            n_trials=OPTUNA_TRIALS,
            timeout=OPTUNA_TIMEOUT,
            show_progress_bar=True
        )
        best_models['CatBoost_Optuna'] = study.best_params
        print(f"    Best MAE: {study.best_value:.2f}")
        print(f"    Best params: {study.best_params}")

In [ ]:
if RUN_OPTUNA and best_models:
    print("\n" + "=" * 60)
    print("EVALUATING OPTUNA-OPTIMIZED MODELS ON TEST SET")
    print("=" * 60)
    
    X_train_full = pd.concat([train, val])[ALL_FEATURES]
    y_train_full = pd.concat([train, val])['count']
    
    for name, params in best_models.items():
        t0 = time.time()
        
        if 'XGBoost' in name:
            model = XGBRegressor(**params, random_state=42, n_jobs=-1, verbosity=0)
        elif 'LightGBM' in name:
            model = LGBMRegressor(**params, random_state=42, n_jobs=-1, verbose=-1)
        elif 'CatBoost' in name:
            model = CatBoostRegressor(**params, random_state=42, verbose=0)
        
        model.fit(X_train_full, y_train_full)
        y_pred = np.clip(model.predict(X_test), 0, None)
        elapsed = time.time() - t0
        
        m = calc_metrics(y_test.values, y_pred, y_test.max())
        beach_df = eval_per_beach(test, y_pred, 'beach')
        avg_rel = beach_df['RelMAE'].mean()
        
        all_results.append({
            'Model': name, 'Type': 'Optuna',
            'MAE': m['MAE'], 'RMSE': m['RMSE'], 'R2': m['R2'],
            'AvgRelMAE': avg_rel, 'Time': elapsed
        })
        print(f"  {name:20s} | {elapsed:5.1f}s | MAE={m['MAE']:.1f} | RelMAE={avg_rel:.1f}% | R2={m['R2']:.3f}")

In [ ]:
def create_lstm_objective(nf_train, nf_val, hist_exog, horizon):
    def objective(trial):
        params = {
            'h': horizon,
            'input_size': INPUT_SIZE,
            'encoder_hidden_size': trial.suggest_int('hidden_size', 32, 256),
            'encoder_n_layers': trial.suggest_int('n_layers', 1, 3),
            'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
            'dropout_prob_theta': trial.suggest_float('dropout', 0.0, 0.5),
            'max_steps': 200,
            'early_stop_patience_steps': -1,
            'batch_size': BATCH_SIZE,
            'scaler_type': 'robust',
            'random_seed': 42,
            'accelerator': ACCELERATOR,
            'devices': DEVICES,
            'hist_exog_list': hist_exog,
            'loss': MAE(),
        }
        
        model = LSTM(**params)
        nf = NeuralForecast(models=[model], freq='h')
        
        nf_all = pd.concat([nf_train, nf_val]).sort_values(['unique_id', 'ds']).reset_index(drop=True)
        cv_results = nf.cross_validation(df=nf_all, n_windows=1, step_size=horizon)
        
        pred_col = [c for c in cv_results.columns if c not in ['unique_id', 'ds', 'cutoff', 'y']][0]
        y_true = cv_results['y'].values
        y_pred = np.clip(cv_results[pred_col].values, 0, None)
        
        return mean_absolute_error(y_true, y_pred)
    return objective

if RUN_OPTUNA and HAS_NF:
    print("\n" + "=" * 60)
    print("OPTUNA LSTM OPTIMIZATION")
    print("=" * 60)
    
    # Prepare smaller dataset for LSTM optimization
    nf_tr, _, _ = prepare_nf_data(train, val)
    nf_va, _, _ = prepare_nf_data(val, test)
    
    horizon_opt = min(24, nf_va.groupby('unique_id').size().min())
    
    print(f"  Optimizing LSTM (horizon={horizon_opt})...")
    study = optuna.create_study(direction='minimize')
    
    try:
        study.optimize(
            create_lstm_objective(nf_tr, nf_va, ALL_FEATURES, horizon_opt),
            n_trials=min(10, OPTUNA_TRIALS),
            timeout=OPTUNA_TIMEOUT,
            show_progress_bar=True
        )
        
        print(f"    Best MAE: {study.best_value:.2f}")
        print(f"    Best params: {study.best_params}")
        best_models['LSTM_Optuna'] = study.best_params
    except Exception as e:
        print(f"    ERROR: {e}")

In [ ]:
if RUN_OPTUNA and HAS_NF and 'LSTM_Optuna' in best_models:
    print("\n  Evaluating LSTM_Optuna on test set...")
    
    params = best_models['LSTM_Optuna']
    horizon = min(72, nf_test.groupby('unique_id').size().min())
    
    t0 = time.time()
    model = LSTM(
        h=horizon,
        input_size=INPUT_SIZE,
        encoder_hidden_size=params.get('hidden_size', 64),
        encoder_n_layers=params.get('n_layers', 2),
        learning_rate=params.get('learning_rate', LEARNING_RATE),
        dropout_prob_theta=params.get('dropout', 0.1),
        max_steps=MAX_STEPS,
        early_stop_patience_steps=-1,
        batch_size=BATCH_SIZE,
        scaler_type='robust',
        random_seed=42,
        accelerator=ACCELERATOR,
        devices=DEVICES,
        hist_exog_list=ALL_FEATURES,
        loss=MAE(),
    )
    
    nf = NeuralForecast(models=[model], freq='h')
    nf_all = pd.concat([nf_train, nf_test]).sort_values(['unique_id', 'ds']).reset_index(drop=True)
    cv_results = nf.cross_validation(df=nf_all, n_windows=1, step_size=horizon)
    elapsed = time.time() - t0
    
    pred_col = [c for c in cv_results.columns if c not in ['unique_id', 'ds', 'cutoff', 'y']][0]
    y_true = cv_results['y'].values
    y_pred = np.clip(cv_results[pred_col].values, 0, None)
    
    m = calc_metrics(y_true, y_pred, y_true.max())
    eval_df = cv_results.copy()
    eval_df['beach'] = eval_df['unique_id']
    beach_df = eval_per_beach(eval_df, y_pred, 'beach')
    avg_rel = beach_df['RelMAE'].mean()
    
    all_results.append({
        'Model': 'LSTM_Optuna', 'Type': 'Optuna+NF',
        'MAE': m['MAE'], 'RMSE': m['RMSE'], 'R2': m['R2'],
        'AvgRelMAE': avg_rel, 'Time': elapsed
    })
    print(f"    {elapsed:.1f}s | MAE={m['MAE']:.1f} | RelMAE={avg_rel:.1f}% | R2={m['R2']:.3f}")

## Results

In [ ]:
results_df = pd.DataFrame(all_results)

save_dir = Path(SAVE_DIR)
save_dir.mkdir(parents=True, exist_ok=True)
results_df.to_csv(save_dir / 'results.csv', index=False)

print("\n" + "=" * 70)
print("RESULTS SUMMARY")
print("=" * 70)

display_df = results_df.sort_values('AvgRelMAE')
print(display_df[['Model', 'Type', 'MAE', 'R2', 'AvgRelMAE', 'Time']].to_string(index=False))

In [ ]:
if len(results_df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Sort by RelMAE
    sorted_df = results_df.dropna().sort_values('AvgRelMAE')
    colors = {'Sklearn': 'steelblue', 'NeuralForecast': 'coral', 'Optuna': 'green', 'Optuna+NF': 'purple'}
    bar_colors = [colors.get(t, 'gray') for t in sorted_df['Type']]
    
    # RelMAE bar chart
    axes[0].barh(sorted_df['Model'], sorted_df['AvgRelMAE'], color=bar_colors)
    axes[0].set_xlabel('Avg RelMAE (%)')
    axes[0].set_title('Model Performance (lower is better)')
    axes[0].invert_yaxis()
    
    # R2 bar chart
    sorted_r2 = results_df.dropna().sort_values('R2', ascending=False)
    bar_colors_r2 = [colors.get(t, 'gray') for t in sorted_r2['Type']]
    axes[1].barh(sorted_r2['Model'], sorted_r2['R2'], color=bar_colors_r2)
    axes[1].set_xlabel('R²')
    axes[1].set_title('R² Score (higher is better)')
    axes[1].invert_yaxis()
    
    # Legend
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor=c, label=t) for t, c in colors.items()]
    fig.legend(handles=legend_elements, loc='upper right')
    
    plt.tight_layout()
    plt.savefig(save_dir / 'comparison.png', dpi=150)
    plt.show()

In [ ]:
print("\n" + "=" * 70)
print("BEST MODEL")
print("=" * 70)

valid_results = results_df.dropna(subset=['AvgRelMAE'])
if len(valid_results) > 0:
    best = valid_results.loc[valid_results['AvgRelMAE'].idxmin()]
    print(f"\nBest: {best['Model']} ({best['Type']})")
    print(f"  MAE: {best['MAE']:.2f}")
    print(f"  RelMAE: {best['AvgRelMAE']:.1f}%")
    print(f"  R²: {best['R2']:.3f}")
    print(f"  Time: {best['Time']:.1f}s")
    
    if 'Optuna' in best['Model'] and best['Model'].replace('_Optuna', '') in best_models:
        print(f"\nBest hyperparameters:")
        for k, v in best_models[best['Model']].items():
            print(f"  {k}: {v}")